In [ ]:
import subprocess
'''initialize fuzzers'''
under_test = {'gif': [6], "pcap": [5], "avi": [6], "zip": [5], "jpg": [1], "midi": [7], "mp4": [1], "png": [5]}

for subject in under_test.keys():
    subprocess.run(['./ffcompile', './templates/'+subject+'.bt', './'+subject+'.cpp'], stdout=subprocess.PIPE)
    subprocess.run(['make', subject+'-fuzzer'], stdout=subprocess.PIPE)
    subprocess.run('g++ -O3 --coverage '+subject+'.cpp fuzzer.cpp -o '+subject+'-fuzzer-coverage -lz', shell=True)

In [ ]:
import subprocess
import math
'''Test used to collect the k-path coverage'''
under_test = {'gif': [6], "pcap": [5], "avi": [6], "zip": [5], "jpg": [1], "midi": [7], "mp4": [1], "png": [5]}
results_dict = {'gif': {}, "pcap": {}, "avi": {}, "zip": {}, "jpg": {}, "midi": {}, "mp4": {}, "png": {}}
test_amount = 1

for subject in under_test.keys():
    print("Initialized "+subject)
    for val in range(1, under_test[subject][0]+1):
        covered_0K_total = 0
        covered_FF_Input_total = 0
        covered_FF_Time_total = 0
        inputs_0K_average = 0
        max_paths = 0
        for i in range(test_amount):
            test = subprocess.run(['./'+subject+'-fuzzer', 'test_k_paths_ids', str(val), subject], stdout=subprocess.PIPE, text=True)
            results = test.stdout.split()
            covered_0K_total += int(results[3].split("/")[0])
            inputs_0K = int(results[5])
            inputs_0K_average +=inputs_0K
            if max_paths == 0:
                max_paths = int(results[3].split("/")[1].split(",")[0])
            covered_FF_Input_total += int(results[9].split("/")[0])
            covered_FF_Time_total += int(results[12].split("/")[0])
            inputs_FF_Time = int(results[14])
        covered_0K_average = (int)(covered_0K_total/test_amount)
        covered_FF_Time_average = (int)(covered_FF_Time_total/test_amount)
        covered_FF_Input_average = (int)(covered_FF_Input_total/test_amount)
        results_dict[subject][val] = [max_paths, covered_0K_average, covered_FF_Input_average, covered_FF_Time_average]
        print('Results, K = '+ str(val) +' K-path: ' + str(covered_0K_average) + '/'+str(max_paths)+' FormatFuzzer Input: '+str(covered_FF_Input_average) + '/'+str(max_paths) +' FormatFuzzer Time: '+str(covered_FF_Time_average) + '/'+str(max_paths))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
'''K-path coverage written down, wanted to rerun them a few more times after collecting the first batch
so I wrote the values used for the graphs down'''
results_dict = {'gif': {1: [86, 86, 66, 84], 2: [97, 97, 75, 94], 3: [100, 100, 72, 96], 4: [82, 82, 60, 81], 5: [48, 48, 28, 47], 6: [14, 14, 7, 13]}, 
                "pcap": {1: [58, 56, 52, 58], 2: [84, 80, 68, 82], 3: [86, 82, 74, 84], 4: [64, 60, 48, 61], 5: [10, 10, 7, 9]},
                "avi": {1: [138, 127, 114, 136], 2: [155, 130, 115, 137], 3: [179, 123, 108, 130], 4: [208, 82, 73, 89], 5: [192, 41, 39, 54], 6: [192, 0, 0, 0]},
                "zip": {1: [126, 119, 69, 112], 2: [164, 156, 92, 150], 3: [164, 154, 81, 151], 4: [91, 85, 35, 75], 5: [12, 12, 0, 6]},
                "jpg": {1: [197, 183, 58, 161]},
                "midi": {1: [68, 54, 48, 67], 2: [76, 63, 50, 73], 3: [75, 65, 45, 72], 4: [73, 61, 49, 70], 5: [64, 53, 35, 60], 6: [53, 41, 31, 49], 7: [8, 3, 1, 4]},
                "mp4": {1: [366, 318, 219, 350]},
                "png": {1: [103, 73, 71, 102], 2: [109, 76, 74, 108], 3: [107, 72, 75, 106], 4: [85, 59, 55, 84], 5: [11, 0, 0, 11]}}

for key in results_dict.keys():
    ax = plt.subplot(111)
    x = 0
    for k in results_dict[key].keys():
        height = results_dict[key][k]
        ax.bar(x-3, height[0], width=2, color='r', align='center')
        ax.bar(x-1, height[1], width=2, color='g', align='center')
        ax.bar(x+1, height[2], width=2, color='c', align='center')
        ax.bar(x+3, height[3], width=2, color='b', align='center')
        x += 10 

    ax.title.set_text(key+ " k-path coverage")
    colors = {'Total paths for given k':'red',
              'Average amount of k-paths covered by 0KFuzzer':'green',
              'Average amount of k-paths covered by FormatFuzzer\n(given the same amount of inputs)':'cyan',
              'Average amount of k-paths covered by FormatFuzzer\n(given the same amount of time)':'blue'}         
    labels = list(colors.keys())
    handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
    plt.legend(handles, labels, loc='lower right', bbox_to_anchor=(0.8, -0.6))
    plt.ylabel('Number of k-paths')
    plt.xlabel('Value of k')
    plt.xticks([a for a in range(x) if a%10 == 0], results_dict[key].keys())
    plt.show()

In [ ]:
import subprocess
import math
'''Test used to measure template and code coverage'''
under_test = {'gif': [6, "gif2png _", "./Test_Subjects/gif2png-v.2.5.14"], "pcap": [5, "tcpdump-4.99.1/tcpdump -nr - < _", "./Test_Subjects/pcapTest"], 
              "avi": [6, "ffmpeg -y -i _ out.mp4", "./Test_Subjects/ffmpeg-4.4"], "zip": [5, 'unzip -P "" -t _', './Test_Subjects/unzip60'],
              "jpg": [1, "build/djpeg _", "./Test_Subjects/libjpeg-turbo-master"], "midi": [7, "timidity/timidity - -o - -Ow < _", "./Test_Subjects/TiMidity++-2.14.0"],
              "mp4": [1, "ffmpeg -y -i _ out.mp4", "./Test_Subjects/ffmpeg-4.4"], "png": [5, "contrib/libtests/readpng < _", "./Test_Subjects/libpng1.6-master"]}
results_dict_subject = {'gif': [], "pcap": [], "avi": [], "zip": [], "jpg": [], "midi": [], "mp4": [], "png": []}
results_dict_template = {'gif': [], "pcap": [], "avi": [], "zip": [], "jpg": [], "midi": [], "mp4": [], "png": []}
test_amount = 20
for subject in under_test.keys():
    percent_0K_average = 0
    percent_0K_Template = 0
    percent_FF_Input_average = 0
    percent_FF_Input_Template = 0
    percent_FF_Time_average = 0
    percent_FF_Time_Template = 0
    for i in range(test_amount):
        print(subject+ ', '+str(i)+" starting")
        if under_test[subject][1] != "None":
            subprocess.run('lcov -z -d .', shell=True, capture_output = True)
            number_of_inputs = subprocess.run(['./'+subject+'-fuzzer-coverage', 'test_k_paths_cov', 'k-path', '1', str(under_test[subject][0]), subject], stdout=subprocess.PIPE, text=True).stdout.split()
            temp = subprocess.run('gcovr --exclude-directories Test_Subjects', shell=True, stdout=subprocess.PIPE, text=True).stdout.split()
            percent_0K_Template += float(temp[temp.index(subject+'.cpp')+3].split('%')[0])
            subprocess.run('(cd '+under_test[subject][2]+' && lcov -z -d .)', shell=True, capture_output = True)
            taken_time = float(number_of_inputs[(under_test[subject][0]*2)+2])
            total_inputs = 0
            for j in range(1, under_test[subject][0]+1):
                cur_inputs = int(number_of_inputs[(j*2)-1])
                total_inputs += cur_inputs
                for inputs in range(1, cur_inputs):
                    command = under_test[subject][1].replace("_", 'K'+ str(j)+ 'Input'+ str(inputs) + '.'+subject)
                    run = subprocess.Popen(under_test[subject][2]+"/"+command, shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
                    try:
                        outs, err = run.communicate(timeout = 1)
                    except subprocess.TimeoutExpired:
                        run.kill()
            subprocess.run('lcov -c -d '+ under_test[subject][2]+' -o cov.info', shell = True, capture_output = True)
            cov_data_total_percent = subprocess.run('genhtml cov.info -o '+ subject +'OKResults', shell = True, capture_output = True).stdout.split()
            res = float(str(cov_data_total_percent[len(cov_data_total_percent)-11]).split('\'')[1].split('%')[0])
            print("0KCov: "+ str(res))
            percent_0K_average += res

            '''Next up the FF results, for the same amount of inputs'''
            subprocess.run('lcov -z -d .', shell=True, capture_output = True)
            subprocess.run(['./'+subject+'-fuzzer-coverage', 'test_k_paths_cov', 'FF_Input', str(total_inputs), subject], stdout=subprocess.PIPE, text=True).stdout.split()
            temp = subprocess.run('gcovr --exclude-directories Test_Subjects', shell=True, stdout=subprocess.PIPE, text=True).stdout.split()
            percent_FF_Input_Template += float(temp[temp.index(subject+'.cpp')+3].split('%')[0])
            subprocess.run('(cd '+under_test[subject][2]+' && lcov -z -d .)', shell=True, capture_output = True)
            for inputs in range(1, total_inputs):
                command = under_test[subject][1].replace("_", 'FF_Input'+ str(inputs) + '.'+subject)
                run = subprocess.Popen(under_test[subject][2]+"/"+command, shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
                try:
                    outs, err = run.communicate(timeout = 1)
                except subprocess.TimeoutExpired:
                    run.kill()
            subprocess.run('lcov -c -d '+ under_test[subject][2]+' -o cov.info', shell = True, capture_output = True)
            cov_data_total_percent = subprocess.run('genhtml cov.info -o '+ subject +'FFResults', shell = True, capture_output = True).stdout.split()
            res = float(str(cov_data_total_percent[len(cov_data_total_percent)-11]).split('\'')[1].split('%')[0])
            print("InputCov: "+ str(res))
            percent_FF_Input_average += res

            '''Lastly FF results for the same amount of time given for the generation'''
            subprocess.run('lcov -z -d .', shell=True, capture_output = True)
            number_of_inputs = subprocess.run(['./'+subject+'-fuzzer-coverage', 'test_k_paths_cov', 'FF_Time', str(taken_time), subject], stdout=subprocess.PIPE, text=True).stdout.split()
            temp = subprocess.run('gcovr --exclude-directories Test_Subjects', shell=True, stdout=subprocess.PIPE, text=True).stdout.split()
            percent_FF_Time_Template += float(temp[temp.index(subject+'.cpp')+3].split('%')[0])
            subprocess.run('(cd '+under_test[subject][2]+' && lcov -z -d .)', shell=True, capture_output = True)
            for inputs in range(1, int(number_of_inputs[1])):
                command = under_test[subject][1].replace("_", 'FF_Time'+ str(inputs) + '.'+subject)
                run = subprocess.Popen(under_test[subject][2]+"/"+command, shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
                try:
                    outs, err = run.communicate(timeout = 1)
                except subprocess.TimeoutExpired:
                    run.kill()
            subprocess.run('lcov -c -d '+ under_test[subject][2]+' -o cov.info', shell = True, capture_output = True)
            cov_data_total_percent = subprocess.run('genhtml cov.info -o '+ subject +'FFResultsTime', shell = True, capture_output = True).stdout.split()
            res = float(str(cov_data_total_percent[len(cov_data_total_percent)-11]).split('\'')[1].split('%')[0])
            print("TimeCov: "+ str(res))
            percent_FF_Time_average += res

    results_dict_subject[subject] = [percent_0K_average/test_amount, percent_FF_Input_average/test_amount, percent_FF_Time_average/test_amount]
    results_dict_template[subject] = [percent_0K_Template/test_amount, percent_FF_Input_Template/test_amount, percent_FF_Time_Template/test_amount]
    print(subject + " finished")
    print("Subject coverage: ")
    print(results_dict_subject)
    print("Template coverage: ")
    print(results_dict_template)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Results from 20 runs, written down due to not having ran them all at once
subject_coverage = {'gif': [35.895, 41.47, 60.645], 'pcap': [2.88, 2.875, 4.51],
                    'zip': [22.145, 21.51, 31.4], 'jpg': [15.475, 15.08, 18.815],
                    'mp4': [4.24, 4.17, 5.58], 'midi': [14.0, 14.02, 16.81],
                    'avi': [3.09, 3.31, 5.78],'png': [12.3, 12.6, 16.72]}
template_coverage = {'gif': [89.5, 87.9, 91.0], 'pcap': [86.05, 86.1, 87.0],
                     'zip': [76.15, 67.1, 81.1], 'jpg': [53.51, 37.45, 79.2], 
                     'mp4': [81.75, 64.25, 92.85], 'midi': [87.85, 86.3, 89.0],
                     'avi': [83.9, 81.25, 87.0], 'png': [78.8, 80.15, 85.0]}

for key in subject_coverage.keys():
    print("Subject coverage: ")
    vals = subject_coverage[key]
    ax = plt.subplot(111)
    x = 0
    ax.bar(x-2, vals[0], width=2, color='g', align='center')
    ax.bar(x, vals[1], width=2, color='c', align='center')
    ax.bar(x+2, vals[2], width=2, color='b', align='center')
    x += 10 

    ax.title.set_text(key+ " subject coverage")
    colors = {'Average code coverage obtained by 0KFuzzer':'green',
              'Average code coverage obtained by FormatFuzzer\n(given the same amount of inputs)':'cyan',
              'Average code coverage obtained by FormatFuzzer\n(given the same amount of time)':'blue'}          
    labels = list(colors.keys())
    handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
    plt.legend(handles, labels, loc='lower right', bbox_to_anchor=(0.8, -0.45))
    plt.ylabel('Code coverage (percentage)')
    plt.tick_params(
        axis = 'x',
        which = 'both',
        bottom = False,
        top = False,
        labelbottom=False)
    plt.show()
    
    print("Template coverage")
    vals = template_coverage[key]
    ax = plt.subplot(111)
    x = 0
    ax.bar(x-2, vals[0], width=2, color='g', align='center')
    ax.bar(x, vals[1], width=2, color='c', align='center')
    ax.bar(x+2, vals[2], width=2, color='b', align='center')
    x += 10 

    ax.title.set_text(key+ " template coverage")
    colors = {'Average template coverage obtained by 0KFuzzer':'green',
              'Average template coverage obtained by FormatFuzzer\n(given the same amount of inputs)':'cyan',
              'Average template coverage obtained by FormatFuzzer\n(given the same amount of time)':'blue'}         
    labels = list(colors.keys())
    handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
    plt.legend(handles, labels, loc='lower right', bbox_to_anchor=(0.85, -0.45))
    plt.ylabel('Code coverage (percentage)')
    plt.tick_params(
        axis = 'x',
        which = 'both',
        bottom = False,
        top = False,
        labelbottom=False)
    plt.show()

In [ ]:
import subprocess
import math
'''Test used to generate the lcov files to compare what parts of the templates and programs were covered'''
under_test = {'gif': [6, "gif2png _", "./Test_Subjects/gif2png-v.2.5.14"], "pcap": [5, "tcpdump-4.99.1/tcpdump -nr - < _", "./Test_Subjects/pcapTest"], 
              "avi": [6, "ffmpeg -y -i _ out.mp4", "./Test_Subjects/ffmpeg-4.4"], "zip": [5, 'unzip -P "" -t _', './Test_Subjects/unzip60'],
              "jpg": [1, "build/djpeg _", "./Test_Subjects/libjpeg-turbo-master"], "midi": [7, "timidity/timidity - -o - -Ow < _", "./Test_Subjects/TiMidity++-2.14.0"],
              "mp4": [1, "ffmpeg -y -i _ out.mp4", "./Test_Subjects/ffmpeg-4.4"], "png": [5, "contrib/libtests/readpng < _", "./Test_Subjects/libpng1.6-master"]}
results_dict_subject = {'gif': [], "pcap": [], "avi": [], "zip": [], "jpg": [], "midi": [], "mp4": [], "png": []}
results_dict_template = {'gif': [], "pcap": [], "avi": [], "zip": [], "jpg": [], "midi": [], "mp4": [], "png": []}
test_amount = 1
for subject in under_test.keys():
    for i in range(test_amount):
        print(subject+ ', '+str(i)+" starting")
        if under_test[subject][1] != "None":
            subprocess.run('lcov -z -d .', shell=True, capture_output = True)
            number_of_inputs = subprocess.run(['./'+subject+'-fuzzer-coverage', 'test_k_paths_cov', 'k-path', '1', str(under_test[subject][0]), subject], stdout=subprocess.PIPE, text=True).stdout.split()
            subprocess.run('lcov -c --directory . -o cov.info --no-external', shell = True, capture_output = True)
            cov_data_total_percent = subprocess.run('genhtml cov.info -o '+ subject +'OKTemplateGeneration', shell = True,capture_output = True)
            subprocess.run('lcov -z -d .' , shell=True, capture_output = True)
            taken_time = float(number_of_inputs[(under_test[subject][0]*2)+2])
            total_inputs = 0
            for j in range(1, under_test[subject][0]+1):
                cur_inputs = int(number_of_inputs[(j*2)-1])
                total_inputs += cur_inputs
                for inputs in range(1, cur_inputs):
                    command = './'+subject+'-fuzzer-coverage parse K'+ str(j)+ 'Input'+ str(inputs) + '.'+subject
                    subprocess.run(command, shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
            subprocess.run('lcov -c --directory . -o cov.info --no-external', shell = True, capture_output = True)
            cov_data_total_percent = subprocess.run('genhtml cov.info -o '+ subject +'OKTemplateCoverage', shell = True, capture_output = True)

            '''Next up the FF results, for the same amount of inputs'''
            subprocess.run('lcov -z -d .', shell=True, capture_output = True)
            subprocess.run(['./'+subject+'-fuzzer-coverage', 'test_k_paths_cov', 'FF_Input', str(total_inputs), subject], stdout=subprocess.PIPE, text=True).stdout.split()
            subprocess.run('lcov -z -d .', shell=True, capture_output = True)
            for inputs in range(1, total_inputs):
                command = './'+subject+'-fuzzer-coverage parse FF_Input'+ str(inputs) + '.'+subject
                subprocess.run(command, shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
            subprocess.run('lcov -c --directory . -o cov.info --no-external', shell = True, capture_output = True)
            cov_data_total_percent = subprocess.run('genhtml cov.info -o '+ subject +'FFTemplateCoverage', shell = True, capture_output = True)
            
            '''Lastly FF results for the same amount of time given for the generation'''
            subprocess.run('lcov -z -d .', shell=True, capture_output = True)
            number_of_inputs = subprocess.run(['./'+subject+'-fuzzer-coverage', 'test_k_paths_cov', 'FF_Time', str(taken_time), subject], stdout=subprocess.PIPE, text=True).stdout.split()
            subprocess.run('lcov -z -d .', shell=True, capture_output = True)
            for inputs in range(1, int(number_of_inputs[1])):
                command = './'+subject+'-fuzzer-coverage parse FF_Time'+ str(inputs) + '.'+subject
                subprocess.run(command, shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
            subprocess.run('lcov -c --directory . -o cov.info --no-external', shell = True, capture_output = True)
            cov_data_total_percent = subprocess.run('genhtml cov.info -o '+ subject +'FFTimeTemplateCoverage', shell = True, capture_output = True)